In [2]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"

if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API Key (optional):"
    )

if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt="Enter your LangSmith Project Name (default = 'default'):"
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"

if not os.environ.get("COHERE_API_KEY"):
  os.environ["COHERE_API_KEY"] = getpass.getpass("Enter API key for Cohere: ")

In [3]:
from langchain.chat_models import init_chat_model

model = init_chat_model("command-r-plus", model_provider="cohere")

In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hi Bob! It's nice to meet you. I'm Coral, an AI-assistant chatbot trained to assist human users like yourself. How can I help you today?", additional_kwargs={'id': 'e514d2ae-da15-49f1-9306-b5af17f3823d', 'finish_reason': 'COMPLETE', 'content': "Hi Bob! It's nice to meet you. I'm Coral, an AI-assistant chatbot trained to assist human users like yourself. How can I help you today?", 'token_count': {'input_tokens': 71.0, 'output_tokens': 34.0}}, response_metadata={'id': 'e514d2ae-da15-49f1-9306-b5af17f3823d', 'finish_reason': 'COMPLETE', 'content': "Hi Bob! It's nice to meet you. I'm Coral, an AI-assistant chatbot trained to assist human users like yourself. How can I help you today?", 'token_count': {'input_tokens': 71.0, 'output_tokens': 34.0}}, id='run--387bd0ac-87d7-404d-9836-44aeab94745f-0', usage_metadata={'input_tokens': 71, 'output_tokens': 34, 'total_tokens': 105})

In [ ]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I don't have personal information about you, so I don't know your name. Is there anything else I can help you with?", additional_kwargs={'id': '2e033321-3370-4453-9df2-d1cb0322cfb1', 'finish_reason': 'COMPLETE', 'content': "I don't have personal information about you, so I don't know your name. Is there anything else I can help you with?", 'token_count': {'input_tokens': 71.0, 'output_tokens': 27.0}}, response_metadata={'id': '2e033321-3370-4453-9df2-d1cb0322cfb1', 'finish_reason': 'COMPLETE', 'content': "I don't have personal information about you, so I don't know your name. Is there anything else I can help you with?", 'token_count': {'input_tokens': 71.0, 'output_tokens': 27.0}}, id='run--e96c271a-f524-46e5-8730-108c24294435-0', usage_metadata={'input_tokens': 71, 'output_tokens': 27, 'total_tokens': 98})

In [5]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?")
    ]
)

AIMessage(content='Your name is Bob. Is there anything else I can help you with?', additional_kwargs={'id': '7c20e97e-eb35-4914-a13f-922b5da4ac9a', 'finish_reason': 'COMPLETE', 'content': 'Your name is Bob. Is there anything else I can help you with?', 'token_count': {'input_tokens': 92.0, 'output_tokens': 15.0}}, response_metadata={'id': '7c20e97e-eb35-4914-a13f-922b5da4ac9a', 'finish_reason': 'COMPLETE', 'content': 'Your name is Bob. Is there anything else I can help you with?', 'token_count': {'input_tokens': 92.0, 'output_tokens': 15.0}}, id='run--fd2380f3-d4ee-481a-be13-497c7fb13d99-0', usage_metadata={'input_tokens': 92, 'output_tokens': 15, 'total_tokens': 107})

In [6]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages" : response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable" : {"thread_id": "abc123"}}

In [ ]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hello, Bob! It's a pleasure to meet you. I'm Coral, an AI-assistant chatbot trained to assist users like you by providing helpful and thorough responses. How can I help you today?


In [ ]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob. Is there anything else I can help you with?


In [ ]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't have personal information about you, so I don't know your name. Is there anything else I can help you with?


In [ ]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob. Is there something else I can help you with?


In [ ]:
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't have personal information about you, so I don't know your name. Is there anything else I can help you with?


In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
# output["messages"][-1].pretty_print()

In [ ]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Yer name be Jim, matey. Now, what can I do fer ye?


In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language} Only. Do not use any other language.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [9]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

workflow = StateGraph(state_schema=State)

def call_model(state: State):
    prompt_messages = prompt_template.format_messages(
        language=state["language"],
        messages=state["messages"]
    )
    response = model.invoke(prompt_messages)
    return {
        "messages": [response],
        "language": state["language"]  # ← Add this line
    }


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id":"abc456"}}
query = "Hi! I'm Bob."
language = "spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hola, soy tu asistente. ¿En qué te puedo ayudar?


In [ ]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bob. ¿Hay algo más en lo que pueda ayudarte?


In [10]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [11]:
workflow = StateGraph(state_schema=State)

def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt_messages = prompt_template.format_messages(
        language=state["language"],
        messages=trimmed_messages
    )
    response = model.invoke(prompt_messages)
    return {
        "messages": [response],
        "language": state["language"]  # ← Add this line
    }


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [12]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arrrr! Ye be nameless, matey!


In [13]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arr, ye asked what be the answer to two plus two, matey!


In [7]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages"
):
    if isinstance(chunk, AIMessage):# Filter to just model responses
        print(chunk.content, end="|")
    

Hello| Todd|!| Why| did| the| chicken| cross| the| playground|?| 

To| get| to| the| other| slide|!| 

I| hope| that| brought| a| smile| to| your| face|,| Todd|.| Would| you| like| to| hear| another| joke|?||